In [1]:
# D=0.25

In [2]:
#!pip install pulp
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from itertools import product
from tqdm import tqdm
import pandas as pd
from pulp import *
import numpy as np

#import files from github
V = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/V.csv') #vehicle types
F = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/F.csv') #fuel types
E = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/E.csv') #driving env
R = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/R.csv') #counties
M = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/M.csv') #charging stations
S = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/S.csv') #states

F['fuel_type'] = F['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))

VEHICLE_TYPES = list(V['vehicle_type'])
FUEL_TYPES = list(F['fuel_type'])
DRIVING_ENV = list(E['driving_environment'])
COUNTIES = list(R['county'])
CHARGING_STATIONS = list(M['filling_stations'])
STATES = list(S['state'])

EF = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/EF(f%2Cs).csv')
EF['fuel_type'] = EF['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
FE = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/FE(v%2Cf).csv')
FE['fuel_type'] = FE['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
C = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/C(F).csv')
CC = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/CC(v%2Cs).csv') 
CC['yearly_cost']=np.where(CC['vehicle_type']=='BEV', CC['cost_minus_rebate']/8.5, CC['cost_minus_rebate']/5.5)
CG = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/CG(F).csv')
CG['fuel_type'] = CG['fuel_type'].apply(lambda x: x.replace('electricity','Electricity'))
# D = 0.25
TM = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/TM(f%2Cs).csv')
N = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/N(r).csv')
B = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/B(r).csv')
CF = FE
CF['fuel_consumption'] = (1 / CF['fuel_economy'])
T = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/T(r).csv')
T['total_vehicles_registered'] = T['total_vehicles_registered'].apply(lambda x: x.replace(',',''))
T['total_vehicles_registered'] = T['total_vehicles_registered'].apply(pd.to_numeric)
W_param = pd.read_csv('https://raw.githubusercontent.com/saif1457/biofuels/master/optimisation_data/W_county_param.csv')

e85_vi = pd.read_csv('e85_vi.csv')
efuels_vi = pd.read_csv('efuels_vi.csv')
e85_vi = e85_vi[['STATE','NAME','CENSUSAREA','e85_area']]
e85_vi = e85_vi.rename(columns={'STATE': 'state', 'NAME': 'county','CENSUSAREA':'census_area'})
efuels_vi = efuels_vi[['STATE','NAME','CENSUSAREA','efuels_area']]
efuels_vi = efuels_vi.rename(columns={'STATE': 'state', 'NAME': 'county','CENSUSAREA':'census_area'})

import feather
readFrame = pd.read_feather('/Users/saifbhatti/Desktop/biofuels/analysed_vdf', use_threads=True)
efuels_vi.county = efuels_vi.county.str.replace('_',' ')
efuels_vi = efuels_vi.merge(readFrame,left_on='county',right_on='County')
efuels_vi = efuels_vi[['state', 'county', 'census_area_x', 'efuels_area_y']]
efuels_vi.rename(columns={'efuels_area_y':'efuels_area','census_area_x':'census_area'},inplace=True)


e85_vi['e85_area'] = e85_vi['e85_area'] / 100
efuels_vi['efuels_area'] = efuels_vi['efuels_area']/100

e85vi_param = e85_vi.groupby('county')['e85_area'].apply(list).to_dict()
efuelsvi_param = efuels_vi.groupby('county')['efuels_area'].apply(list).to_dict()

KeyError: "['CENSUSAREA', 'STATE', 'NAME'] not in index"

In [ ]:
efuels_vi[efuels_vi['county']=='San Francisco']

In [ ]:
efuels_vi.to_csv('efuels_vi.csv')

In [ ]:
efuels_vi.describe()

In [ ]:
e85_vi[e85_vi['county']=='San Francisco']

In [ ]:
# e85_vi.to_csv('e85_vi.csv')

In [ ]:
e85_vi.describe()

In [ ]:
electric = pd.read_csv('/Users/saifbhatti/Desktop/Northwestern/sy1920/s20/iems394/client-data/iems394/preprocessed_data/electric_fuel_stations.csv')

In [ ]:
electric = electric[['County','State']]

In [ ]:
electric = electric[electric['State'].isin(['MN','TX','CA'])]

In [ ]:
repeated_list = ['Orange','Cass','Lake','Trinity','Houston','Polk','Brown','Clay','Jackson','Washington','Martin']
electric.loc[electric['County'].isin(repeated_list), 'County'] = electric['County'] + ' ' + electric['State']



In [ ]:
electric['Electric Station Count'] = electric.groupby(by='County').count()['State']

In [ ]:
electric.groupby(by='County').count().reset_index(name='County')

In [ ]:
electric['Efuel Station Count'] = electric.groupby('County')['County'].transform('count')

In [ ]:
electric = electric.groupby(by=['County','State']).count()

In [ ]:
electric.reset_index(inplace=True)
electric.to_csv('efuel_station_count')

In [ ]:
electric